In [ ]:
from datetime import date
import os
from collections import defaultdict
import re

In [ ]:
###cell for file directories

def exists(direc, file_name):
    exists_ornah = os.path.exists(direc)
    while exists_ornah == False:
        #print('Attempted directory: ' + os.getcwd()+ '\n')
        if os.path.isdir(direc) == True:
            direc = input("Directory does not exist.. please enter directory, i.e '/home/name/Dektop/BDD2204 for the file " + file_name + ': \n')
        elif os.path.isdir(direc) == False:
            direc = input("File does not exist.. please enter file directory, i.e '/home/name/Dektop/BDD2204 for the file " + file_name + ': \n')

        direc = os.path.normpath(direc)
        exists_ornah = os.path.exists(direc)
        print('Attempted directory: ' + os.getcwd()+ '\n')
        
    print('Current directory: ' + os.getcwd() +'/n')
    return(direc)

parent_dir = input("Enter file director, i.e '/home/name/Dektop/BDD1724\n")
parent_dir = exists(parent_dir, 'BDD174, general folder')


Enter file director, i.e '/home/name/Dektop/BDD1724
/home/name/Dektop/BDD2204
File does not exist.. please enter file directory, i.e '/home/name/Dektop/BDD2204 for the file BDD174, general folder: 
/home/name/Dektop/BDD2204/
Attempted directory: /home/ale/notebooks/immport_ipython-master



#### JIRA BDD-1724

Goal: Curation of clinical data

In [ ]:
#day0file = "/home/cthomas/Desktop/cris/projects/BDD1724/files/clinical_data_SDY369_372_376.txt"
#outputfile = "/home/cthomas/Desktop/cris/projects/BDD1724/files/deID_clinical_data_SDY369_372_376.txt"
day0file = parent_dir + 'files/clinical_data_SDY369_372_376.txt' 
day0file = exists(day0file, 'folder containing clinical_data_SDY369_372_376.txt: clinical data?')

outputfile = parent_dir + 'files/' 
outputfile = exists(day0file, 'deID_clinical_data_SDY369_372_376.txt: clinical data?')

## De-identify dates:
# Find day 0
day0 = {}     # day0[SUBJECT_USER_DEFINED_ID] = DAY 0 in MM/DD/YYYY 
dates = defaultdict(dict)    # dates[SUBJECT_USER_DEFINED_ID][USER_DEF_PLANNED_VISIT_ID] = DATE in MM/DD/YYYY 
with open(day0file, "r") as day0f:
    fl = day0f.readline()
    
    for lines in day0f:
        elemts = lines.split("\t")
        if elemts[2].startswith("D0"):
            day0[elemts[0]] = elemts[3]
            dates[elemts[0]][elemts[2]] = elemts[3]
        else:
            dates[elemts[0]][elemts[2]] = elemts[3]
        
deIDdates = defaultdict(dict)
# Re-calculate dates
for subjects in dates:
    for planned_visits in dates[subjects]:
        mh0, dy0, yr0 = day0[subjects].split("/")
        mh, dy, yr = dates[subjects][planned_visits].split("/")
        newdate = date(int(yr), int(mh), int(dy)) - date(int(yr0), int(mh0), int(dy0))
        deIDdates[subjects][planned_visits] = str(newdate.days)



In [ ]:
## Get SUBJECTS ACC instead of user defined.
## Ran query 923 to get mapping between user defined ID of subjects and ACC_NUM
#mappingIDs = "/home/cthomas/Desktop/cris/projects/BDD1724/files/cthomas_923_2779_userdefIDs-ACC.BDD1724.txt"
mappingIDs = parent_dir + 'files/cthomas_923_2779_userdefIDs-ACC.BDD1724.txt'
mappingIDs = exists(mappingIDs, 'cthomas_923_2779_userdefIDs-ACC.BDD1724.txt: subbjects acc of user defined')

subIDs = {}

with open(mappingIDs, "r") as mpids:
    ff = mpids.readline()
    
    for ligne in mpids:
        ids = ligne.split("\t")
        subIDs[ids[1]] = ids[0]

In [ ]:
## re-print inpt
with open(day0file, "r") as inf, open(outputfile, "w") as outf:
    headings = inf.readline()
    outf.write(headings)
    
    for alllines in inf:
        col = alllines.split("\t")
        col[3] = deIDdates[col[0]][col[2]]
        col[0] = subIDs[col[0]]
        outf.write("\t".join(col))


In [ ]:
## Prepare table for transformation tool - assessment page.
## need to add assessment panel ID, assessment name, assessment component ID, age at onset + unit
#agesfile = "/home/cthomas/Desktop/cris/projects/BDD1724/files/subjects_age_BDD1724.txt"
agesfile = parent_dir + 'files/subjects_age_BDD1724.txt'
agesfile = exists(agesfile, 'subjects_age_BDD1724.txt: file of ages?')
#assessmtfile = "/home/cthomas/Desktop/cris/projects/BDD1724/files/subjects_assessments_BDD1724.txt"
assessmtfile = parent_dir + 'subjects_assessments_BDD1724.txt'
assessmtfile = exists(assessmtfile, 'subjects_assessments_BDD1724.txt: file of the subjects assessments?')

# output file
#prepTTassessments = "/home/cthomas/Desktop/cris/projects/BDD1724/files/prepTTassessments.txt"
prepTTassessments = parent_dir + 'files/prepTTassessments'
prepTTassessments = exists(prepTTassessments, 'prepTTassessments.txt: output file for assesments')


## get age of subjects
ages = {}
with open(agesfile, "r") as af:
    fff = af.readline()
    for subj in af:
        subj = subj.strip()
        subjectsAges = subj.split("\t")
        ages[subIDs[subjectsAges[0]]] = subjectsAges[2]
        
## generate input file for transformation tool
with open(assessmtfile, "r") as asfl, open(prepTTassessments, "w") as prpTT:
    rer = asfl.readline()
    rer = rer.strip()

    newheadings = [
        "panel_id",
        "comp_id_cmas", 
        "comp_id_mdglob", 
        "comp_id_mmt8", 
        "comp_id_symptoms", 
        "age", 
    ]
    addon = "\t".join(newheadings)
    prpTT.write("\t".join([rer, addon]))
    prpTT.write("\n")
        
    for lignes in asfl:
        lignes = lignes.strip("\n")
        assmts = lignes.split("\t")
        panelID = "_".join(["HIPCBaylor", assmts[1], "physical_exam"])
        compIDcmas = "_".join(["HIPCBaylor", assmts[1], "CMAS"])
        compIDmmt8 = "_".join(["HIPCBaylor", assmts[1], "MMT8"])
        compIDglob = "_".join(["HIPCBaylor", assmts[1], "MD_Global"])
        compIDsympt = "_".join(["HIPCBaylor", assmts[1], "Symptoms"])
        newcols = "\t".join([panelID, compIDcmas, compIDglob, compIDmmt8, compIDsympt, ages[assmts[0]]])
        prpTT.write("\t".join([lignes, newcols]))
        prpTT.write("\n")

In [ ]:
## INTERVENTIONS

#interprep = "/home/cthomas/Desktop/cris/projects/BDD1724/files/subjects_interventions_BDD1724.txt"
interpret = parent_dir + 'files/subjects_interventions_BDD1724.txt'
interpret = exists(interpret, 'subjects_interventions_BDD1724.txt: yeah no hints here...')

#interventions = "/home/cthomas/Desktop/cris/projects/BDD1724/files/interventions.txt"
suffixcount = {"SDY369":0, "SDY372":0, "SDY376":0}
empty = ""

with open(interprep, "r") as ip, open(interventions, "w") as iv:
    ## prep template:
    iv.write("\t".join(["interventions","Schema Version 2.30","Content Version 2.30"]))
    iv.write("\n")
    iv.write("Please do not delete or edit this column\n")
    iv.write("\t".join(["Column Name","Intervention User-Defined ID","Subject ID","Study ID",
                        "Name","Compound Name","Compound Role","Dose Reported","Start Day",
                        "End Day","Status","Indication","Formulation","Dose","Dose Units",
                        "Dose Frequency per Interval","Route of Administration","Is Ongoing",
                        "Start Time","End Time","Start Day From Partial Date",
                        "End Day From Partial Date","Duration","Duration Unit"]))
    iv.write("\n")
    
    ## get data to write
    ipfl = ip.readline()
    for ipline in ip:
        ipline = ipline.strip("\n")
        ipstuff = ipline.split("\t")
        suffixcount[ipstuff[1]] += 1
        doseReported = "UNK"
        if re.match("\d", ipstuff[7]):
            doseReported = " ".join([ipstuff[7], ipstuff[8]])
        interID = "_".join(["HIPCBaylor", ipstuff[1], str(suffixcount[ipstuff[1]]).zfill(3)])

        newstring = "\t".join([empty, interID, ipstuff[0], ipstuff[1]])
        newstring1 = "\t".join([ipstuff[5],ipstuff[5],"Concomitant Medication",doseReported,ipstuff[3]])
        newstring2 = "\t".join([empty, empty, ipstuff[10], empty, ipstuff[7], ipstuff[8]])
        newstring3 = "\t".join([ipstuff[9], ipstuff[11]])
        newstring4 = "\t" * 6
        iv.write("\t".join([newstring, newstring1, newstring2, newstring3, newstring4]))
        iv.write("\n")
    
    
    